In [69]:
from dotenv import load_dotenv
import os
from web3 import Web3
from eth_account import Account
import pandas as pd
import time
import random
import math
from diskcache import Cache
import datetime as dt
from datetime import timedelta
from uniswap import Uniswap
import json
from pydantic import BaseModel, Field
from typing import List
import numpy as np
from openai import OpenAI

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")
FRED_API_KEY = os.getenv('FRED_API_KEY')

ACCOUNT_ADDRESS = os.getenv('ACCOUNT_ADDRESS')
PRIVATE_KEY = os.getenv('PRIVATE_KEY')
GATEWAY = os.getenv('ARBITRUM_GATEWAY')

In [3]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
os.chdir('..')

In [5]:
from python_scripts.web3_utils import *
from python_scripts.apis import token_classifier_portfolio, flipside_api_results
from python_scripts.macro_data import main as macro_main
from sql_queries.queries import latest_portfolio_metrics
from python_scripts.utils import fetch_and_process_tbill_data, prepare_data_for_simulation

2025-01-11 18:50:08.042 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Current Directory: e:\Projects\encode hackathon


In [6]:
cache = Cache('classifier_data')

In [12]:
cache_dict = {key: cache[key] for key in cache}
print(cache_dict)

{'arbitrum_classifier params': {'model': 'arbitrum_classifier', 'days': 7, 'top': 20, 'network': 'arbitrum', 'backtest_period': 4380, 'start_date': '2024-07-13'}, 'arbitrum_classifier_portfolio':   symbol                               token_address  sharpe_ratio  \
0   LAVA  0x11e969e9b3f89cb16d686a03cd8508c9fc0361af     31.123394   
1  MAGIC  0x539bde0d7dbd336b79148aa742883198bbf60342      4.465459   
2    STG  0x6694340fc020c5e6b96567843da2df01b2ce1eb6      4.262890   
3   USDC  0xaf88d065e77c8cc2239327c5edb3a432268e5831      3.525502   
4    DAI  0xda10009cbd5d07dd0cecc66161fc93d7c9000da1      3.177627   
5   USDC  0xff970a61a04b1ca14834a43f5de4533ebddb5cc8      2.392183   
6    ARB  0x912ce59144191c1204e64559fe8253a0e49e6548      2.121842   
7   USDT  0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9      0.905344   
8    CRV  0x11cdb42b0eb46d95f990bedd4695a6e3fa034978      0.605335   
9    ZRO  0x6985884c4392d348587b19cb9eaaf157f13271cd      0.244965   

   excess_return  latest_price   

In [13]:
historical_data = cache.get(f'historical_data', pd.DataFrame())
historical_port_values = cache.get(f'historical_port_values', pd.DataFrame())
oracle_prices = cache.get(f'oracle_prices',pd.DataFrame())
last_rebalance_time = cache.get(f'last_rebalance_time', None)
model_actions = cache.get(f'actions', pd.DataFrame()) 

In [14]:
def update_historical_data(live_comp):
    global historical_data
    new_data = pd.DataFrame([live_comp])
    historical_data = pd.concat([historical_data, new_data]).reset_index(drop=True)
    historical_data.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'historical_data', historical_data)

def update_portfolio_data(values):
    global historical_port_values
    print(f'values: {values}')
    values = pd.DataFrame([values])
    historical_port_values = pd.concat([historical_port_values, values]).reset_index(drop=True)
    historical_port_values.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'historical_port_values', historical_port_values)

def update_price_data(values):
    global oracle_prices

    # Ensure the 'hour' column exists by resetting index if necessary
    if isinstance(values.index, pd.DatetimeIndex):
        values = values.reset_index().rename(columns={'index': 'hour'})
    
    if 'hour' not in values.columns:
        raise ValueError("The provided DataFrame must have a 'hour' column.")

    # Concatenate the new values with the existing oracle_prices
    oracle_prices = pd.concat([oracle_prices, values]).drop_duplicates(subset='hour', keep='last').reset_index(drop=True)
    
    # Cache the updated oracle_prices
    cache.set(f'oracle_prices', oracle_prices)

    print(f'Updated oracle_prices:\n{oracle_prices}')

def update_model_actions(actions):
    global model_actions
    print(f'model actions before update: {model_actions}')
    new_data = pd.DataFrame(actions)
    print(f'new data: {new_data}')
    model_actions = pd.concat([model_actions, new_data]).reset_index(drop=True)
    model_actions.drop_duplicates(subset='Date', keep='last', inplace=True)
    cache.set(f'actions', model_actions)

In [15]:
class CompositionItem(BaseModel):
    asset: str = Field(..., description="The cryptocurrency symbol, e.g., ETH, BTC.")
    weight: float = Field(..., description="Fraction of total portfolio in [0,1]. The sum of all weights should be 1.")
    reasoning: str = Field(..., description="A brief explanation for why this allocation is chosen.")

class Rebalance(BaseModel):
    target_composition: List[CompositionItem] = Field(
        default_factory=list,
        description="A list of target allocations for each asset."
    )

class PositionReasoning(BaseModel):
    """
    This model wraps the Rebalance object under 'rebalance'.
    """
    rebalance: Rebalance = Field(
        default_factory=Rebalance,
        description="All target composition details."
    )

In [37]:
def network_def(chain):
        if chain == 'gnosis':
            primary_gateway = GNOSIS_GATEWAY  # Replace with your Infura URL
            backup_gateway = 'https://lb.nodies.app/v1/406d8dcc043f4cb3959ed7d6673d311a'  # Your backup gateway
        elif chain == 'arbitrum':
            primary_gateway = GATEWAY  # Replace with your Infura URL
            backup_gateway = GATEWAY
        elif chain == 'optimism':
            primary_gateway = OPTIMISM_GATEWAY  # Replace with your Infura URL
            backup_gateway = OPTIMISM_GATEWAY
        elif chain == 'ethereum':
            primary_gateway = ETHEREUM_GATEWAY  # Replace with your Infura URL
            backup_gateway = ETHEREUM_GATEWAY

        print(f'Gateway: {primary_gateway}')

        for gateway in [primary_gateway, backup_gateway]:
            w3 = Web3(Web3.HTTPProvider(gateway))
            if w3.is_connected():
                try:
                    latest_block = w3.eth.get_block('latest')['number']  # Only try this if connected
                    print(f"Connected to {chain} via {gateway}: {latest_block} block")
                    return w3, gateway
                except Exception as e:
                    print(f"Connected to {gateway} but failed to fetch latest block. Error: {e}")
            else:
                print(f"Failed to connect to {chain} via {gateway}. Trying next gateway...")

        raise ConnectionError(f"Failed to connect to {chain} network using both primary and backup gateways.")

In [17]:
def rebalance_portfolio(
    uniswap, 
    token_contracts, 
    token_decimals, 
    target_compositions, 
    account_address, 
):
    """
    Rebalances the portfolio by selling all tokens into WETH and then buying target allocations using WETH.

    Parameters:
    - uniswap: Initialized Uniswap class instance.
    - token_contracts: Dict of token addresses.
    - token_decimals: Dict of token decimals.
    - target_compositions: Dict of target compositions as fractions summing to 1.
    - account_address: ETH wallet address.
    - web3: Initialized Web3 instance.
    """

    # WETH address and checksum
    WETH_ADDRESS = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
    checksum_weth_address = Web3.to_checksum_address(WETH_ADDRESS)

    # Step 1: Convert Token Addresses to Checksum Format
    checksum_addresses = {token: Web3.to_checksum_address(address) for token, address in token_contracts.items()}

    # Step 2: Sell All Current Token Holdings into WETH
    for token, address in checksum_addresses.items():
        try:
            balance_wei = uniswap.get_token_balance(address)
            balance = balance_wei / 10**token_decimals[token]
            
            # Adjust the balance to avoid precision issues (round down to 6 decimal places)
            adjusted_balance = math.floor(balance * 10**8) / 10**8
            
            if adjusted_balance > 0:
                amount_to_sell = int(adjusted_balance * 10**token_decimals[token])
                print(f"Selling {adjusted_balance:.6f} {token} for WETH")
                uniswap.make_trade(
                    checksum_addresses[token],
                    checksum_weth_address,  # WETH as output token
                    amount_to_sell
                )
                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)
        except Exception as e:
            print(f"Error selling {token}: {e}")

    # Step 3: Get Current WETH Balance
    weth_balance_wei = uniswap.get_token_balance(checksum_weth_address)
    weth_balance = weth_balance_wei / 10**18
    print(f"Total WETH balance after selling: {weth_balance:.6f} WETH")

    # Step 4: Buy Target Tokens Based on Target Compositions
    for token, target_weight in target_compositions.items():
        if target_weight > 0:
            weth_to_spend = weth_balance * target_weight
            
            # Adjust the WETH amount to avoid precision issues (round down to 6 decimal places)
            adjusted_weth_to_spend = math.floor(weth_to_spend * 10**8) / 10**8

            if adjusted_weth_to_spend <= 0:
                continue

            try:
                print(f"Buying {token} with {adjusted_weth_to_spend:.6f} WETH")

                uniswap.make_trade(
                    checksum_weth_address,        # WETH as input token
                    checksum_addresses[token],    # Target token
                    int(adjusted_weth_to_spend * 10**18),  # Convert WETH amount to wei
                    fee=3000                      # Assuming 0.3% fee pool for Uniswap V3
                )

                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)

            except Exception as e:
                print(f"Error buying {token}: {e}")

    # Step 5: Log the Rebalancing Info
    final_weth_balance = uniswap.get_token_balance(checksum_weth_address) / 10**18
    print(f"Final WETH balance: {final_weth_balance:.6f} WETH")

    rebal_info = {
        "account_address": account_address,
        "initial_weth_balance": weth_balance,
        "final_weth_balance": final_weth_balance,
        "purchases": target_compositions,
    }

    # Save rebalancing info to CSV
    rebal_df = pd.DataFrame([rebal_info])
    rebal_df.to_csv('data/live_rebal_results.csv', index=False)
    print("Rebalancing info saved to 'data/live_rebal_results.csv'.")

In [18]:
# import json

# # Define the path to your ABI file
# mlAMPL_abi_file_path = "abi/mlAMPL.json"
# WETH_abi_file_path = "abi/weth_abi.json"
# bonding_abi_file_path = "abi/bonding_contract_abi.json"
# lp_manager_path = 'abi/nftmanager.json'
# staking_path = 'abi/staking_abi.json'
# erc20_abi_path = 'abi/erc20_abi.json'
# nft_factory_path = 'abi/nft_factory.json'
# pool_abi_path = 'abi/pool_abi.json'

# abi_paths = [mlAMPL_abi_file_path, WETH_abi_file_path, bonding_abi_file_path,lp_manager_path,staking_path,erc20_abi_path,nft_factory_path,pool_abi_path]
# abis = {}

# for path in abi_paths:
#     with open(path, "r") as file:
#         abis[path] = json.load(file)

# print(abis)


In [19]:
def data_cleaning(df,dropna=True,ffill=False):
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        # clean_df = clean_df.dropna(axis=1, how='any')
        clean_df = clean_df.dropna()
    if ffill == True:
        clean_df = clean_df.resample('h').ffill().bfill()

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    prices_df = prices_df.drop_duplicates(subset=['hour', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='hour',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['hour'] + [f'{col}_Price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [20]:
def pull_data(function,path,model_name, api=False,start_date=None):
    print(f'function:{function},start_date:{start_date},path:{path},api:{api},model_name: {model_name}')

    if api:
        print(f'api True')
        # Parse dates into datetime format for consistency
        start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        
        # Use formatted date strings as needed in the dao_advisor_portfolio and lst_portfolio_prices functions
        prices = function(start_date.strftime('%Y-%m-%d %H:%M:%S'))
        
        prices_df = flipside_api_results(prices, flipside_api_key)

        prices_df.to_csv(path)
    else:
        print(f'api False')
        prices_df = pd.read_csv(path)

    dataset = {
        f'portfolio': prices_df
    }

    return dataset

In [21]:
def prices_data_func(network,
                     api_key,use_cached_data,name,days=None,
                     function=None,start_date=None,
                     backtest_period=None,filtered_assets=None):
    
    if start_date is None and backtest_period is None:
        raise KeyError("Provide either a start date or backtest_period")
    
    print(f"backtest days: {(pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days}")
    
    if backtest_period is None:
        backtest_period = (pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days * 24
        if backtest_period < 1:
            backtest_period = 1

    if function is None:

        data = token_classifier_portfolio(
            network=network,
            days=days,
            name=name,
            api_key = api_key,
            use_cached_data=use_cached_data,
            start_date = start_date,
            prices_only=True
        )

        prices_df = data_cleaning(data['portfolio'])
        prices_df
    else: 
        data = pull_data(function=function,start_date=start_date, path=f'data/{name}.csv', api=not use_cached_data,model_name=name)
        prices_df = data_cleaning(data['portfolio'])
        prices_df = prices_df[prices_df.index >= start_date].dropna()
        prices_df

    # prices_df.columns = prices_df.columns.str.replace('_Price','')
    filtered_assets_with_price = [f"{asset}_Price" for asset in filtered_assets]


    return data, prices_df[filtered_assets_with_price]

In [22]:
def prepare_data_for_simulation(price_timeseries, start_date, end_date):
    """
    Ensure price_timeseries has entries for start_date and end_date.
    If not, fill in these dates using the last available data.
    """
    # Ensure 'ds' is in datetime format
    # price_timeseries['hour'] = pd.to_datetime(price_timeseries['hour'])
    
    # Set the index to 'ds' for easier manipulation
    # if price_timeseries.index.name != 'hour':
    #     price_timeseries.set_index('hour', inplace=True)

    print(f'price index: {price_timeseries.index}')

    price_timeseries.index = price_timeseries.index.tz_localize(None)
    
    # Check if start_date and end_date exist in the data
    required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
    all_dates = price_timeseries.index.union(required_dates)
    
    # Reindex the dataframe to ensure all dates from start to end are present
    price_timeseries = price_timeseries.reindex(all_dates)
    
    # Forward fill to handle NaN values if any dates were missing
    price_timeseries.fillna(method='ffill', inplace=True)

    # Reset index if necessary or keep the datetime index based on further needs
    # price_timeseries.reset_index(inplace=True, drop=False)
    # price_timeseries.rename(columns={'index': 'hour'}, inplace=True)
    # price_timeseries.set_index('hour',inplace=True)
    
    return price_timeseries


In [28]:
network = 'arbitrum'

model = f'{network}_classifier'

params = cache.get(f'{model} params')
classifier_data = cache.get(f'{model}_portfolio')
original_prices_df = cache.get(f'{model}_prices')
days = params['days']

In [35]:
#With this, we can calculate excess return, stdev, and sharpe over correct range; using prices_df with this start date

(dt.datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d %H:00:00')

'2025-01-04 18:00:00'

In [60]:
chain = params['network']

w3, gateway = network_def(chain)

account = Account.from_key(PRIVATE_KEY)
w3.eth.default_account = account.address

three_month_tbill_historical_api = "https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&file_type=json"

try: 
    three_month_tbill = fetch_and_process_tbill_data(api_url=three_month_tbill_historical_api, api_key=FRED_API_KEY,
                                                     data_key="observations",
                                                       date_column="date", 
                                                       value_column="value")
    three_month_tbill['decimal'] = three_month_tbill['value'] / 100
    current_risk_free = three_month_tbill['decimal'].iloc[-1]
    print(f"3-month T-bill data fetched: {three_month_tbill.tail()}")
except Exception as e:
    print(f"Error in fetching tbill data: {e}")

Gateway: https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl
Connected to arbitrum via https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl: 294468690 block
3-month T-bill data fetched:            realtime_start realtime_end  value  decimal
date                                                  
2024-08-01     2025-01-09   2025-01-09   5.05   0.0505
2024-09-01     2025-01-09   2025-01-09   4.72   0.0472
2024-10-01     2025-01-09   2025-01-09   4.51   0.0451
2024-11-01     2025-01-09   2025-01-09   4.42   0.0442
2024-12-01     2025-01-09   2025-01-09   4.27   0.0427


In [39]:
network = chain

In [41]:
full_start = original_prices_df.index.min().strftime('%Y-%m-%d %H:00:00')
full_start

'2025-01-11 14:00:00'

In [42]:
filtered_assets = classifier_data['symbol'].unique()
filtered_assets

array(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'],
      dtype=object)

In [43]:
data_start_date = dt.datetime.now(dt.timezone.utc) - timedelta(days=days)
data_start_date = data_start_date.strftime('%Y-%m-%d %H:00:00')

today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

data_version = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H-00-00')
data_version_comp = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

In [44]:
# Importing macro data from helper function

interest_rate_dict, cpi_data_df_clean_prepared = macro_main(formatted_today_utc, api=False)

Loading data from CSVs...


In [45]:
interest_rate_dict['Overnight Interbank Rate']

,date,value,country
0,2024-12-01,0.0448,united_states


In [46]:
cpi_data_df_clean_prepared

,country,value,expenditure
0,united_states,0.027494,total


In [47]:
start_date = str(data_start_date)
end_date = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

start_date

'2025-01-04 23:00:00'

In [48]:
end_date

'2025-01-11 23:00:00'

In [49]:
days = params['days']

In [50]:
data, prices_df = prices_data_func(
                            network=network, 
                            name=model,
                            api_key=flipside_api_key,
                            use_cached_data=False,
                            function=None,
                            start_date=start_date,
                            filtered_assets=filtered_assets
                            )
        
prices_df = prepare_data_for_simulation(prices_df, start_date, end_date)

backtest days: 7
use_cached_data: False
volume_threshold: 1
start_date: 2025-01-04 23:00:00
data_start_str: 2024-07-08 04:00:00
Beginning: '2025-01-04 23:00:00'
Query not completed. Retrying in 30 seconds...
cleaning prices
cleaned prices:                          hour  ARB_Price  CRV_Price  DAI_Price  LAVA_Price  \
0    2025-01-04T23:00:00.000Z   0.899137   1.053000   1.002000    0.176141   
1    2025-01-05T00:00:00.000Z   0.895715   1.057000   1.001000    0.175642   
2    2025-01-05T01:00:00.000Z   0.896103   1.049000   1.003000    0.174869   
3    2025-01-05T02:00:00.000Z   0.887224   1.049000   0.999907    0.174444   
4    2025-01-05T03:00:00.000Z   0.879932   1.037000   0.999887    0.172983   
..                        ...        ...        ...        ...         ...   
164  2025-01-11T19:00:00.000Z   0.729853   0.800173   1.001000    0.155837   
165  2025-01-11T20:00:00.000Z   0.727764   0.801735   1.000000    0.153570   
166  2025-01-11T21:00:00.000Z   0.727122   0.802024   0.99

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_2208\3188632879.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_2208\3188632879.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)


In [63]:
prices_df.columns = [col.replace('_Price', '') for col in prices_df.columns]
prices_returns = prices_df.pct_change().dropna()
prices_returns

,LAVA,MAGIC,STG,USDC,DAI,ARB,USDT,CRV,ZRO
2025-01-05 00:00:00,-0.002833,0.005610,-0.004239,0.000887,-0.000998,-0.003806,0.000293,0.003799,-0.001733
2025-01-05 01:00:00,-0.004401,-0.018491,0.008741,0.001000,0.001998,0.000433,0.000281,-0.007569,0.000000
2025-01-05 02:00:00,-0.002430,-0.014069,0.087233,-0.000999,-0.003084,-0.009908,0.000106,0.000000,-0.008681
2025-01-05 03:00:00,-0.008375,0.019930,-0.042328,-0.000521,-0.000020,-0.008219,-0.000017,-0.011439,-0.012259
2025-01-05 04:00:00,-0.004446,-0.030821,0.016018,0.000481,-0.002260,0.000173,0.001023,-0.001929,0.007092
...,...,...,...,...,...,...,...,...,...
2025-01-11 19:00:00,0.005854,-0.007456,-0.009507,0.000802,0.001000,-0.004731,0.000321,-0.007870,-0.004320
2025-01-11 20:00:00,-0.014547,-0.000896,-0.009406,-0.000002,-0.000999,-0.002862,0.000738,0.001952,0.000000
2025-01-11 21:00:00,-0.003008,-0.001885,-0.006155,0.000003,-0.000071,-0.000882,0.001071,0.000360,-0.002169
2025-01-11 22:00:00,-0.001247,0.025293,0.007341,0.003000,0.002071,0.022277,-0.000999,0.045535,0.021739


In [52]:
metrics_query = latest_portfolio_metrics(classifier_data['token_address'], network, days, dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d'))

In [53]:
latest_metrics_df = flipside_api_results(metrics_query, flipside_api_key)
latest_metrics_df

Query not completed. Retrying in 30 seconds...


,symbol,token_address,sharpe_ratio,excess_return,latest_price,latest_hour,sixty_day_price,stddev_30d,rolling_7d_avg,token_return,rolling_30d_avg,avg_vol,sum_vol,__row_index
0,LAVA,0x11e969e9b3f89cb16d686a03cd8508c9fc0361af,None,None,0.153266,2025-01-11T23:00:00.000Z,0.155225,0.036943,0.165106,-0.012620,0.165539,215.330862,3.855930e+06,0
1,MAGIC,0x539bde0d7dbd336b79148aa742883198bbf60342,None,None,0.486079,2025-01-11T23:00:00.000Z,0.536896,0.070258,0.532215,-0.094650,0.536009,288.138052,5.849779e+06,1
2,STG,0x6694340fc020c5e6b96567843da2df01b2ce1eb6,None,None,0.443151,2025-01-11T23:00:00.000Z,0.421014,0.058353,0.450932,0.052580,0.452665,189.897801,2.226932e+06,2
3,USDT,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,None,None,1.001000,2025-01-11T23:00:00.000Z,0.999334,0.001438,0.999975,0.001667,0.999972,1395.118420,6.058985e+08,3
4,USDC,0xaf88d065e77c8cc2239327c5edb3a432268e5831,None,None,0.999900,2025-01-11T23:00:00.000Z,0.999459,0.000841,1.000024,0.000441,1.000010,2010.084340,1.249788e+09,4
5,ZRO,0x6985884c4392d348587b19cb9eaaf157f13271cd,None,None,4.780000,2025-01-11T23:00:00.000Z,5.780000,0.601146,5.179231,-0.173010,5.245365,484.841270,6.153121e+06,5
6,DAI,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,None,None,1.001000,2025-01-11T23:00:00.000Z,0.999981,0.001595,1.000098,0.001019,1.000026,820.129601,1.134977e+07,6
7,USDC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,None,None,1.002000,2025-01-11T23:00:00.000Z,0.998867,0.001698,1.000071,0.003137,1.000050,794.366441,1.608401e+08,7
8,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,None,None,0.860535,2025-01-11T23:00:00.000Z,1.090000,0.113152,0.910250,-0.210518,0.929288,883.966083,6.262900e+06,8
9,ARB,0x912ce59144191c1204e64559fe8253a0e49e6548,None,None,0.756907,2025-01-11T23:00:00.000Z,0.833937,0.097364,0.825650,-0.092369,0.828726,823.798768,2.233796e+08,9


In [61]:
three_month_tbill['hourly_rate'] = (1 + three_month_tbill['decimal']) ** (1 / 8760) - 1

,realtime_start,realtime_end,value,decimal,hourly_rate
date,,,,,
1934-01-01,2025-01-09,2025-01-09,0.72,0.0072,8.189734e-07
1934-02-01,2025-01-09,2025-01-09,0.62,0.0062,7.055778e-07
1934-03-01,2025-01-09,2025-01-09,0.24,0.0024,2.736444e-07
1934-04-01,2025-01-09,2025-01-09,0.15,0.0015,1.711046e-07
1934-05-01,2025-01-09,2025-01-09,0.16,0.0016,1.825025e-07
...,...,...,...,...,...
2024-08-01,2025-01-09,2025-01-09,5.05,0.0505,5.624016e-06
2024-09-01,2025-01-09,2025-01-09,4.72,0.0472,5.264847e-06
2024-10-01,2025-01-09,2025-01-09,4.51,0.0451,5.035695e-06


In [62]:
three_month_tbill = prepare_data_for_simulation(three_month_tbill[three_month_tbill.index>'2024-01-01'], data_start_date, end_date)
three_month_tbill

price index: DatetimeIndex(['2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01',
               '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01',
               '2024-10-01', '2024-11-01', '2024-12-01'],
              dtype='datetime64[ns]', name='date', freq=None)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_2208\3188632879.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_2208\3188632879.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)


,realtime_start,realtime_end,value,decimal,hourly_rate
2024-02-01 00:00:00,2025-01-09,2025-01-09,5.24,0.0524,0.000006
2024-03-01 00:00:00,2025-01-09,2025-01-09,5.24,0.0524,0.000006
2024-04-01 00:00:00,2025-01-09,2025-01-09,5.24,0.0524,0.000006
2024-05-01 00:00:00,2025-01-09,2025-01-09,5.25,0.0525,0.000006
2024-06-01 00:00:00,2025-01-09,2025-01-09,5.24,0.0524,0.000006
...,...,...,...,...,...
2025-01-11 19:00:00,2025-01-09,2025-01-09,4.27,0.0427,0.000005
2025-01-11 20:00:00,2025-01-09,2025-01-09,4.27,0.0427,0.000005
2025-01-11 21:00:00,2025-01-09,2025-01-09,4.27,0.0427,0.000005
2025-01-11 22:00:00,2025-01-09,2025-01-09,4.27,0.0427,0.000005


In [67]:
combined_data = prices_returns.merge(three_month_tbill[['hourly_rate']], left_index=True,right_index=True,how='left')
asset_columns = prices_returns.columns  # List of asset symbols, e.g., ['LAVA', 'MAGIC', ...]
excess_returns = combined_data[asset_columns].subtract(combined_data['hourly_rate'], axis=0)
print(excess_returns.head())



                         LAVA     MAGIC       STG      USDC       DAI  \
2025-01-05 00:00:00 -0.002838  0.005605 -0.004244  0.000882 -0.001003   
2025-01-05 01:00:00 -0.004406 -0.018496  0.008736  0.000995  0.001993   
2025-01-05 02:00:00 -0.002435 -0.014073  0.087228 -0.001004 -0.003089   
2025-01-05 03:00:00 -0.008380  0.019926 -0.042333 -0.000526 -0.000025   
2025-01-05 04:00:00 -0.004450 -0.030826  0.016013  0.000476 -0.002265   

                          ARB      USDT       CRV       ZRO  
2025-01-05 00:00:00 -0.003811  0.000288  0.003794 -0.001738  
2025-01-05 01:00:00  0.000428  0.000276 -0.007573 -0.000005  
2025-01-05 02:00:00 -0.009913  0.000101 -0.000005 -0.008685  
2025-01-05 03:00:00 -0.008224 -0.000022 -0.011444 -0.012264  
2025-01-05 04:00:00  0.000168  0.001018 -0.001933  0.007087  


In [70]:
std_dev = excess_returns.std()

# Mean excess return for each asset
mean_excess_return = excess_returns.mean()

# Hourly Sharpe ratio for each asset
sharpe_ratio = mean_excess_return / std_dev

print("Hourly Sharpe Ratios for each asset:")
print(sharpe_ratio)

annualization_factor = np.sqrt(8760)  # sqrt of hours in a year
annualized_sharpe_ratio = sharpe_ratio * annualization_factor

print("\nAnnualized Sharpe Ratios for each asset:")
print(annualized_sharpe_ratio)

Hourly Sharpe Ratios for each asset:
LAVA    -0.028090
MAGIC   -0.113152
STG     -0.006507
USDC     0.007919
DAI     -0.003618
ARB     -0.105601
USDT     0.003632
CRV     -0.088066
ZRO     -0.108088
dtype: float64

Annualized Sharpe Ratios for each asset:
LAVA     -2.629044
MAGIC   -10.590493
STG      -0.609066
USDC      0.741170
DAI      -0.338654
ARB      -9.883693
USDT      0.339915
CRV      -8.242539
ZRO     -10.116489
dtype: float64


In [75]:
mean_excess_return_df = mean_excess_return.to_frame('Excess Returns')

In [76]:
sharpe_ratios_df = sharpe_ratio.to_frame('Sharpe Ratios')

In [77]:
sharpe_ratios_df

,Sharpe Ratios
LAVA,-0.028090
MAGIC,-0.113152
STG,-0.006507
USDC,0.007919
DAI,-0.003618
ARB,-0.105601
USDT,0.003632
CRV,-0.088066
ZRO,-0.108088


In [78]:
mean_excess_return_df

,Excess Returns
LAVA,-0.000597
MAGIC,-0.001401
STG,-0.000105
USDC,0.000014
DAI,-0.000008
ARB,-0.000986
USDT,0.000007
CRV,-0.001125
ZRO,-0.001075


In [79]:
# Map sharpe_ratios_df values to latest_metrics_df
latest_metrics_df['sharpe_ratio'] = latest_metrics_df['symbol'].map(sharpe_ratios_df['Sharpe Ratios'])

# Map mean_excess_return_df values to latest_metrics_df
latest_metrics_df['excess_return'] = latest_metrics_df['symbol'].map(mean_excess_return_df['Excess Returns'])

In [80]:
latest_metrics_df

,symbol,token_address,sharpe_ratio,excess_return,latest_price,latest_hour,sixty_day_price,stddev_30d,rolling_7d_avg,token_return,rolling_30d_avg,avg_vol,sum_vol,__row_index
0,LAVA,0x11e969e9b3f89cb16d686a03cd8508c9fc0361af,-0.028090,-0.000597,0.153266,2025-01-11T23:00:00.000Z,0.155225,0.036943,0.165106,-0.012620,0.165539,215.330862,3.855930e+06,0
1,MAGIC,0x539bde0d7dbd336b79148aa742883198bbf60342,-0.113152,-0.001401,0.486079,2025-01-11T23:00:00.000Z,0.536896,0.070258,0.532215,-0.094650,0.536009,288.138052,5.849779e+06,1
2,STG,0x6694340fc020c5e6b96567843da2df01b2ce1eb6,-0.006507,-0.000105,0.443151,2025-01-11T23:00:00.000Z,0.421014,0.058353,0.450932,0.052580,0.452665,189.897801,2.226932e+06,2
3,USDT,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,0.003632,0.000007,1.001000,2025-01-11T23:00:00.000Z,0.999334,0.001438,0.999975,0.001667,0.999972,1395.118420,6.058985e+08,3
4,USDC,0xaf88d065e77c8cc2239327c5edb3a432268e5831,0.007919,0.000014,0.999900,2025-01-11T23:00:00.000Z,0.999459,0.000841,1.000024,0.000441,1.000010,2010.084340,1.249788e+09,4
5,ZRO,0x6985884c4392d348587b19cb9eaaf157f13271cd,-0.108088,-0.001075,4.780000,2025-01-11T23:00:00.000Z,5.780000,0.601146,5.179231,-0.173010,5.245365,484.841270,6.153121e+06,5
6,DAI,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,-0.003618,-0.000008,1.001000,2025-01-11T23:00:00.000Z,0.999981,0.001595,1.000098,0.001019,1.000026,820.129601,1.134977e+07,6
7,USDC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,0.007919,0.000014,1.002000,2025-01-11T23:00:00.000Z,0.998867,0.001698,1.000071,0.003137,1.000050,794.366441,1.608401e+08,7
8,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,-0.088066,-0.001125,0.860535,2025-01-11T23:00:00.000Z,1.090000,0.113152,0.910250,-0.210518,0.929288,883.966083,6.262900e+06,8
9,ARB,0x912ce59144191c1204e64559fe8253a0e49e6548,-0.105601,-0.000986,0.756907,2025-01-11T23:00:00.000Z,0.833937,0.097364,0.825650,-0.092369,0.828726,823.798768,2.233796e+08,9


In [81]:
portfolio = classifier_data[['symbol','token_address']]

TOKEN_CONTRACTS = {
    row['symbol']: row['token_address'] for _, row in portfolio.iterrows()
}

TOKEN_DECIMALS = get_token_decimals(TOKEN_CONTRACTS,w3)


TOKEN_CONTRACTS

{'LAVA': '0x11e969e9b3f89cb16d686a03cd8508c9fc0361af',
 'MAGIC': '0x539bde0d7dbd336b79148aa742883198bbf60342',
 'STG': '0x6694340fc020c5e6b96567843da2df01b2ce1eb6',
 'USDC': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc8',
 'DAI': '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1',
 'ARB': '0x912ce59144191c1204e64559fe8253a0e49e6548',
 'USDT': '0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9',
 'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'ZRO': '0x6985884c4392d348587b19cb9eaaf157f13271cd'}

In [82]:
TOKEN_DECIMALS

{'LAVA': 6,
 'MAGIC': 18,
 'STG': 18,
 'USDC': 6,
 'DAI': 18,
 'ARB': 18,
 'USDT': 6,
 'CRV': 18,
 'ZRO': 18}

In [83]:
latest_prices = {
    token: float(prices_df[f"{token}"].iloc[-1])
    for token in TOKEN_CONTRACTS.keys()
    if f"{token}" in prices_df.columns
}

latest_prices

{'LAVA': 0.153266,
 'MAGIC': 0.486079,
 'STG': 0.443151,
 'USDC': 1.002,
 'DAI': 1.001,
 'ARB': 0.756907,
 'USDT': 1.001,
 'CRV': 0.860535,
 'ZRO': 4.78}

In [84]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS,w3)

model_balances  

model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
model_balances_usd

available_balance = sum(model_balances_usd.values())
available_balance

comp_dict = {
    f"{token}": balance_usd / available_balance
    for token, balance_usd in model_balances_usd.items()
}

# comp_dict["date"] = formatted_today_utc

# update_historical_data(comp_dict)

portfolio_dict = {
    "Portfolio Value": available_balance,
    "date": formatted_today_utc
}

# update_portfolio_data(portfolio_dict)

Balances for account 0xFA475d5FB90C4b90D929a64732DA6De38a966416: {'LAVA': 1.21423, 'MAGIC': 0.0, 'STG': 1.2030749924394333, 'USDC': 0.366237, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.0, 'ZRO': 0.0}
balances: dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
TOKEN_CONTRACTS.keys(): dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])


In [85]:
identity_map = {
    "risk_level": "Moderate risk, willing to risk some money for the right investments but not chasing every new opportunity.",
    "token_preferences": "Only tokens appearing in the classifier portfolio.",
    "mission_statement": "Accumulate as much WETH as possible given the available funds.",
}

In [86]:
latest_overnight_interbank_rate = interest_rate_dict['Overnight Interbank Rate']

In [87]:
macro_dict = {}

for key in interest_rate_dict.keys():
    macro_dict[key] = f'{interest_rate_dict[key]["value"].iloc[0] * 100}%'
print(macro_dict)

{'Overnight Interbank Rate': '4.4799999999999995%', '3-Month Rate': '4.46%', '10-Year Rate': '4.38999999999999%'}


In [88]:
macro_dict['cpi'] = f'{cpi_data_df_clean_prepared["value"].iloc[0] * 100}%'

In [89]:
print(macro_dict)

{'Overnight Interbank Rate': '4.4799999999999995%', '3-Month Rate': '4.46%', '10-Year Rate': '4.38999999999999%', 'cpi': '2.74938%'}


In [90]:
macro_summary = ", ".join(
    f"{k}: {macro_dict[k]}"
    for k in macro_dict
)
macro_summary

'Overnight Interbank Rate: 4.4799999999999995%, 3-Month Rate: 4.46%, 10-Year Rate: 4.38999999999999%, cpi: 2.74938%'

In [93]:
data_summary_list = []
for symbol in latest_metrics_df['symbol'].unique():
    row = latest_metrics_df[latest_metrics_df['symbol'] == symbol].iloc[0]
    text = (
        f"{symbol} stats as of {pd.to_datetime(row['latest_hour']).strftime('%Y-%m-%d %H:%M')}: "
        f"Excess Return: {row['excess_return']}, "
        f"sharpe ratio: {row['sharpe_ratio']} "
        f"Latest Price: ${row['latest_price']}, "
        f"{days} day Return: {row['token_return']*100}% "
        f"7 day rolling average: ${row['rolling_7d_avg']} "
        f"30 day rolling average: ${row['rolling_30d_avg']} "
        f"{days} day average volume: ${row['avg_vol']} "
        f"{days} day total volume: ${row['sum_vol']} "
    )
    data_summary_list.append(text)

# Join the list into a single string with newlines
formatted_data_summary = "\n".join(data_summary_list)

In [94]:
formatted_data_summary

'LAVA stats as of 2025-01-11 23:00: Excess Return: -0.0005974983179521281, sharpe ratio: -0.028089618785822064 Latest Price: $0.153266, 7 day Return: -1.262038976% 7 day rolling average: $0.1651064201 30 day rolling average: $0.1655394427 7 day average volume: $215.330862233 7 day total volume: $3855929.75 \nMAGIC stats as of 2025-01-11 23:00: Excess Return: -0.001400975567041897, sharpe ratio: -0.11315249492506303 Latest Price: $0.486079, 7 day Return: -9.464961557% 7 day rolling average: $0.5322146036 30 day rolling average: $0.5360094688 7 day average volume: $288.138051916 7 day total volume: $5849778.73 \nSTG stats as of 2025-01-11 23:00: Excess Return: -0.00010516406696395837, sharpe ratio: -0.006507471628180069 Latest Price: $0.443151, 7 day Return: 5.258019923% 7 day rolling average: $0.4509323669 30 day rolling average: $0.4526647292 7 day average volume: $189.897800802 7 day total volume: $2226931.51 \nUSDT stats as of 2025-01-11 23:00: Excess Return: 7.231607845897032e-06, s

In [95]:
assets = classifier_data['symbol'].unique()
assets

array(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'],
      dtype=object)

In [96]:
recent_headlines = ("*ROBINHOOD: CRYPTO TRADING VOLUMES OVER $30B, UP 600% YOY\n"
                     "*FED’S MUSALEM: TIME MAY BE APPROACHING TO SLOW OR PAUSE RATE CUTS\n")

chat_summary = "Bob is happy with the Ethereum roadmap and has been hearing more people talk about it.\n"

chat_and_data_summary = chat_summary + recent_headlines

In [97]:
user_message = (
    "# Instructions:\n"
    "Here are some details about my trading portfolio. Please help me make decisions to rebalance it based on the provided data.\n"
    "# Personality\n"
    f"{identity_map.get('chat_personality')}\n"
    "# Risk Level\n"
    f"{identity_map.get('risk_level')}\n"
    "This represents the total $USD value of the account, including positions, margin, and available funds.\n"
    "# Available Balance\n"
    f"{available_balance}\n"
    "Portions of this 'available_balance' can be used for placing new orders or modifying existing positions.\n"
    "Always leave a fraction of the total 'available_balance' as a safety buffer for unforeseen volatility.\n"
    "The 'available_balance' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n"
    "# Open Positions\n"
    f"{comp_dict}\n"
    "# Here is the most recent information I want to base my decisions on:\n"
    f"{formatted_data_summary}\n"
    "# Here is additional macro economic data for context\n"
    f"{macro_summary}\n"
    "\n"
    "# Please provide a JSON dictionary matching the following format:\n"
    "```json\n"
    "{\n"
    "  \"target_composition\": [\n"
    "    {\n"
    "      \"asset\": \"ETH\",\n"
    "      \"weight\": 0.50,\n"
    "      \"reasoning\": \"ETH has a strong Sharpe Ratio and good recent performance.\"\n"
    "    },\n"
    "    {\n"
    "      \"asset\": \"BTC\",\n"
    "      \"weight\": 0.25,\n"
    "      \"reasoning\": \"BTC remains a large-cap store of value with moderate performance.\"\n"
    "    },\n"
    "    {\n"
    "      \"asset\": \"ADA\",\n"
    "      \"weight\": 0.25,\n"
    "      \"reasoning\": \"ADA presents an opportunity for growth given recent metrics.\"\n"
    "    }\n"
    "  ]\n"
    "}\n"
    "```\n"
    "\n"
    "### Requirements:\n"
    "- The sum of all `weight` values **must equal 1**.\n"
    "- Provide a brief `reasoning` for each asset.\n"
    "- Provide **only** the JSON response. **No** extra text or explanation outside the JSON.\n"
    "- Each `weight` must be a decimal between 0 and 1.\n"
)


In [98]:
user_message

'# Instructions:\nHere are some details about my trading portfolio. Please help me make decisions to rebalance it based on the provided data.\n# Personality\nNone\n# Risk Level\nModerate risk, willing to risk some money for the right investments but not chasing every new opportunity.\nThis represents the total $USD value of the account, including positions, margin, and available funds.\n# Available Balance\n1.0862135351545272\nPortions of this \'available_balance\' can be used for placing new orders or modifying existing positions.\nAlways leave a fraction of the total \'available_balance\' as a safety buffer for unforeseen volatility.\nThe \'available_balance\' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n# Open Positions\n{\'LAVA\': 0.1713292728888017, \'MAGIC\': 0.0, \'STG\': 0.49082787934389077, \'USDC\': 0.3378428477673076, \'DAI\': 0.0, \'ARB\': 0.0, \'USDT\': 0.0, \'CRV\': 0.0, \'ZRO\': 0.0}\n# H

In [99]:
system_prompt = (
    "# Instructions:\n"
    "Act as a knowledgeable cryptocurrency assistant helping users manage and optimize their trading portfolio.\n"
    "Users understand that trading cryptocurrency is inherently risky and seek to make informed, strategic decisions to maximize their returns.\n"
    "You will be provided with the following data:\n"
    "- `available_balance`: Represents the user's total available USD value for making new trades.\n"
    "- `risk_level`: Indicates the user's risk tolerance (e.g., Low, Moderate, High).\n"
    "- `current_positions`: A dictionary mapping cryptocurrency symbols to their current USD value holdings (e.g., {'ETH': 1500, 'BTC': 1000}).\n"
    "- `market_data`: Recent performance metrics for each asset, including Sharpe Ratio, Excess Return, Latest Price, and 60-Day Return (e.g., {'ETH': {'sharpe_ratio': 1.5, 'excess_return': 0.05, 'latest_price': 3000, '60d_return': 20.0}, ...}).\n"
    "\n"
    "Your primary objective is to provide a rebalance strategy that aims to maximize the portfolio's returns while adhering to the following guidelines:\n"
    "\n"
    "## **Risk Management:**\n"
    "- Ensure that no individual asset exceeds the `available_balance` if you choose to buy or hold more of it.\n"
    "- Maintain a safety buffer within the `available_balance` to handle market volatility and unforeseen events.\n"
    "- Respect the user's `risk_level` by adjusting allocations accordingly.\n"
    "- No asset should be more than 80% of the portfolio.\n"
    "- Avoid allocations below 0.1 USD in absolute terms, to ensure meaningful positions.\n"
    "- There should only be one stablecoin per target composition.  For example, if it is best to include 50% in a stablecoin, choose just one stablecoin."
    "\n"
    "## **Performance Optimization:**\n"
    "- Prioritize assets with higher Sharpe Ratios and Excess Returns to maximize risk-adjusted returns.\n"
    "- Consider reallocating from underperforming assets to those with strong performance metrics.\n"
    "- Ensure diversification to spread risk while seeking high-return opportunities.\n"
    "\n"
    "## **Target Composition:**\n"
    "Instead of listing positions to open, modify, or maintain, **provide a single portfolio composition** where each asset has a `weight` (fraction of total portfolio) and a short `reasoning`.\n"
    "- **Asset**: The cryptocurrency symbol (e.g., ETH, BTC).\n"
    "- **Weight**: A decimal in [0, 1], where the sum of all weights must equal 1.0.\n"
    "- **Reasoning**: A concise explanation for why the asset is allocated that weight.\n"
    "\n"
    "## **Output Format:**\n"
    "Provide a JSON dictionary with the following structure:\n"
    "```json\n"
    "{\n"
    "  \"target_composition\": [\n"
    "    {\n"
    "      \"asset\": \"ETH\",\n"
    "      \"weight\": 0.50,\n"
    "      \"reasoning\": \"ETH has a strong Sharpe Ratio and robust recent performance.\"\n"
    "    },\n"
    "    {\n"
    "      \"asset\": \"BTC\",\n"
    "      \"weight\": 0.30,\n"
    "      \"reasoning\": \"BTC remains a large-cap with moderate performance.\"\n"
    "    },\n"
    "    {\n"
    "      \"asset\": \"SOL\",\n"
    "      \"weight\": 0.20,\n"
    "      \"reasoning\": \"SOL presents growth potential given recent metrics.\"\n"
    "    }\n"
    "  ]\n"
    "}\n"
    "```\n"
    "\n"
    "### Requirements:\n"
    "- All `weight` values **must sum to 1.0**.\n"
    "- Provide a brief `reasoning` for each asset.\n"
    "- **No additional fields** other than `target_composition`.\n"
    "- Provide **only** the JSON response. **No** extra text or explanation outside the JSON.\n"
)


In [100]:
# append our messages to the chat
model = "gpt-4o-2024-08-06"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message},
]


In [101]:
comp_dict

{'LAVA': 0.1713292728888017,
 'MAGIC': 0.0,
 'STG': 0.49082787934389077,
 'USDC': 0.3378428477673076,
 'DAI': 0.0,
 'ARB': 0.0,
 'USDT': 0.0,
 'CRV': 0.0,
 'ZRO': 0.0}

In [102]:
completion = openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=PositionReasoning,
)

try:
    # Access the content of the first choice
    response_content = completion.choices[0].message.content

    # Parse the JSON string into a Python dictionary
    result = json.loads(response_content)

    # Access the rebalance dictionary
    rebalance_dict = result.get("rebalance", {})

    print("Rebalance Strategy:", rebalance_dict)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Raw response:", completion.choices[0].message.content)

Rebalance Strategy: {'target_composition': [{'asset': 'STG', 'weight': 0.45, 'reasoning': 'STG has shown positive 7-day returns despite negative Sharpe Ratio. Offers some growth potential in this portfolio.'}, {'asset': 'USDC', 'weight': 0.4, 'reasoning': 'USDC provides stability with the highest positive Sharpe Ratio and ensures liquidity and volatility buffer.'}, {'asset': 'LAVA', 'weight': 0.15, 'reasoning': 'LAVA is retained at a lower weight due to negative performance metrics, only as a speculative small-cap allocation.'}]}


In [103]:
rebalance_dict['target_composition']

[{'asset': 'STG',
  'weight': 0.45,
  'reasoning': 'STG has shown positive 7-day returns despite negative Sharpe Ratio. Offers some growth potential in this portfolio.'},
 {'asset': 'USDC',
  'weight': 0.4,
  'reasoning': 'USDC provides stability with the highest positive Sharpe Ratio and ensures liquidity and volatility buffer.'},
 {'asset': 'LAVA',
  'weight': 0.15,
  'reasoning': 'LAVA is retained at a lower weight due to negative performance metrics, only as a speculative small-cap allocation.'}]

In [104]:
latest_metrics_df[['symbol','token_address']]

,symbol,token_address
0,LAVA,0x11e969e9b3f89cb16d686a03cd8508c9fc0361af
1,MAGIC,0x539bde0d7dbd336b79148aa742883198bbf60342
2,STG,0x6694340fc020c5e6b96567843da2df01b2ce1eb6
3,USDT,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9
4,USDC,0xaf88d065e77c8cc2239327c5edb3a432268e5831
5,ZRO,0x6985884c4392d348587b19cb9eaaf157f13271cd
6,DAI,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1
7,USDC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8
8,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978
9,ARB,0x912ce59144191c1204e64559fe8253a0e49e6548


In [105]:
factory = '0x1F98431c8aD98523631AE4a59f267346ea31F984'
router = '0x5E325eDA8064b456f4781070C0738d849c824258'
version = 3

In [106]:
uniswap = Uniswap(address=ACCOUNT_ADDRESS, private_key=PRIVATE_KEY, version=version, provider=gateway,router_contract_addr=router,factory_contract_addr=factory)

In [107]:
rebalance_list = rebalance_dict["target_composition"]

In [108]:
new_compositions = {
    token: next(
        (item["weight"] for item in rebalance_list if item["asset"] == token),
        0.0  # default if not found
    )
    for token in TOKEN_CONTRACTS
}

print(new_compositions)

{'LAVA': 0.15, 'MAGIC': 0.0, 'STG': 0.45, 'USDC': 0.4, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.0, 'ZRO': 0.0}


In [109]:
rebalance_portfolio(
    uniswap, 
    TOKEN_CONTRACTS, 
    TOKEN_DECIMALS, 
    new_compositions, 
    ACCOUNT_ADDRESS
)

Selling 1.214230 LAVA for WETH


No fee set, assuming 0.3%


Waiting 29 seconds before the next call...
Selling 1.203075 STG for WETH


Approving 0x6694340fc020c5E6B96567843da2df01b2CE1eb6...
No fee set, assuming 0.3%


Waiting 22 seconds before the next call...
Selling 0.366237 USDC for WETH


Approving 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8...
No fee set, assuming 0.3%


Waiting 20 seconds before the next call...
Total WETH balance after selling: 0.000319 WETH
Buying LAVA with 0.000048 WETH
Waiting 16 seconds before the next call...
Buying STG with 0.000143 WETH
Waiting 21 seconds before the next call...
Buying USDC with 0.000127 WETH
Waiting 26 seconds before the next call...
Final WETH balance: 0.000000 WETH
Rebalancing info saved to 'data/live_rebal_results.csv'.


In [110]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS,w3)

model_balances  

model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
model_balances_usd

available_balance = sum(model_balances_usd.values())
available_balance

comp_dict = {
    f"{token}": balance_usd / available_balance
    for token, balance_usd in model_balances_usd.items()
}

print(f'comp_dict: {comp_dict}')

# comp_dict["date"] = formatted_today_utc

# update_historical_data(comp_dict)

portfolio_dict = {
    "Portfolio Value": available_balance,
    "date": formatted_today_utc
}

print(f'portfolio value: {portfolio_dict}')

# update_portfolio_data(portfolio_dict)

Balances for account 0xFA475d5FB90C4b90D929a64732DA6De38a966416: {'LAVA': 0.185005, 'MAGIC': 0.0, 'STG': 1.1029831848006608, 'USDC': 0.417529, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.0, 'ZRO': 0.0}
balances: dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
TOKEN_CONTRACTS.keys(): dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
comp_dict: {'LAVA': 0.03030973816150359, 'MAGIC': 0.0, 'STG': 0.5224846318077659, 'USDC': 0.4472056300307305, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.0, 'ZRO': 0.0}
portfolio value: {'Portfolio Value': 0.9355071356575976, 'date': '2025-01-11 23:00:00'}
